# Load an Image from an OMERO.server, segment it using StarDist and save the results back to the server.

The notebook shows:

 - How to load an image from an OMERO server.
 - how to segment using a DeepLearning software, [StarDist](https://github.com/stardist/stardist).
 - How to convert the stardist labels into OME ROIs.
 - how to save them back to the OMERO.server.

The image is referenced in the paper "NesSys: a novel method for accurate nuclear segmentation in 3D" published August 2019 in PLOS Biology: https://doi.org/10.1371/journal.pbio.3000388 and can be viewed online in the [Image Data Resource](https://idr.openmicroscopy.org/webclient/?show=image-6001247).


In this notebook, the image is loaded and analyzed using a DeepLearning model software [StarDist](https://github.com/stardist/stardist). The StarDist analysis produces a segmentation, which is then converted and saved to the OMERO.server.

If you wish to run the notebook locally or run the corresponding [Python script](../scripts/idr0062_prediction_save.py), please read instruction in [README](https://github.com/ome/omero-guide-python/blob/master/README.md).

### Install dependencies if required

The cell below will install dependencies if you choose to run the notebook in [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true). 

In [38]:
%pip install omero-py stardist==0.8.1

Note: you may need to restart the kernel to use updated packages.


### Import 

In [39]:
from omero.gateway import BlitzGateway

import matplotlib.pyplot as plt
%matplotlib inline
import numpy

### Create a connection to the OMERO Server

In [3]:
from omero.gateway import BlitzGateway

from getpass import getpass

HOST = 'wss://workshop.openmicroscopy.org/omero-ws'
conn = BlitzGateway(input("Username: "),
                    getpass("OMERO Password: "),
                    host=HOST, secure=True)
conn.connect()

Username: trainer-1
OMERO Password: ········


True

### ID of the image to analyze

In [40]:
image_id = 55506

In [41]:
image = conn.getObject("Image", image_id)
print(image.getName())

B4_C2.tif


### Helper method to load the 5D image

The image is loaded a TCZYX numpy array.

In [42]:
def load_numpy_array(image):
    pixels = image.getPrimaryPixels()
    size_z = image.getSizeZ()
    size_c = image.getSizeC()
    size_t = image.getSizeT()
    size_y = image.getSizeY()
    size_x = image.getSizeX()
    z, t, c = 0, 0, 0  # first plane of the image

    zct_list = []
    for t in range(size_t):
        for c in range(size_c):  # all channels
            for z in range(size_z):  # get the Z-stack
                zct_list.append((z, c, t))

    values = []
    # Load all the planes as YX numpy array
    planes = pixels.getPlanes(zct_list)
    s = "t:%s c:%s z:%s y:%s x:%s" % (size_t, size_c, size_z, size_y, size_x)
    print(s)
    print("Downloading image %s" % image.getName())
    all_planes = numpy.stack(list(planes))
    shape = (size_t, size_c, size_z, size_y, size_x)
    return numpy.reshape(all_planes, newshape=shape)

### Load the binary data
Load the binary data as a numpy array

In [43]:
data = load_numpy_array(image)

t:1 c:2 z:257 y:284 x:281


## Load StarDist trained model 

In [44]:
from stardist.models import StarDist2D
model_versatile = StarDist2D.from_pretrained('2D_demo')

Found model '2D_demo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.486166, nms_thresh=0.5.


## Prediction based on a default StarDist model
Normalize the input image

``model_versatile.predict_instances`` will

 * predict object probabilities and star-convex polygon distances (see model.predict if you want those)
 * perform non-maximum suppression (with overlap threshold nms_thresh) for polygons above object probability threshold prob_thresh.
 * render all remaining polygon instances in a label image
 * return the label instances image and also the details (coordinates, etc.) of all remaining polygons

In [45]:
from csbdeep.utils import normalize
axis_norm = (0,1)
c = 1
img = normalize(data[0, c, :, :, :], 1,99.8, axis=axis_norm)
results = []
results_details = []
for i in range(len(img)):
    new_labels, details = model_versatile.predict_instances(img[i])
    results_details.append(details)
    results.append(new_labels)

label_slices = numpy.array(results)

## Save the StarDist labels

StarDist offers method to save the labels into ImageJ rois using ``export_imagej_rois``. This is outside the scope of this notebook. 

Below we show how to save the segmentation represented as polygon to the OMERO.server.
* Convert the StarDist polygon coordinates into OME Polygons
* Save the output back to the OMERO.server.

In [46]:
# Helper for generating the color integers for shapes
def rgba_to_int(red, green, blue, alpha=255):
    """ Return the color as an Integer in RGBA encoding """
    r = red << 24
    g = green << 16
    b = blue << 8
    a = alpha
    rgba_int = r+g+b+a
    if (rgba_int > (2**31-1)):       # convert to signed 32-bit int
        rgba_int = rgba_int - 2**32
    return rgba_int

In [ ]:
# Convert into OME Polygons
import omero
c = 1

rois = []
for i in range(len(results_details)):
    details = results_details[i]
    roi = omero.model.RoiI()
    roi.setImage(image._obj)
    for obj_id, region in enumerate(details['coord']):
        coordinates = []
        x = region[1]
        y = region[0]
        for j in range(len(x)):
            coordinates.append(str(float(x[j])) + "," + str(float(y[j])))
        joined_string = ", ". join(coordinates)
        polygon = omero.model.PolygonI()
        polygon.theZ = omero.rtypes.rint(i)
        polygon.theC = omero.rtypes.rint(c)
        polygon.fillColor = omero.rtypes.rint(rgba_to_int(255, 0, 255, 50))
        polygon.strokeColor = omero.rtypes.rint(rgba_to_int(255, 255, 0))
        polygon.strokeWidth = omero.model.LengthI(10, omero.model.enums.UnitsLength.PIXEL)
        polygon.points = omero.rtypes.rstring(joined_string)
        polygon.textValue = omero.rtypes.rstring("stardist")
        roi.addShape(polygon)
        rois.append(roi)
conn.getUpdateService().saveCollection(rois)

### Close the connection  

In [ ]:
conn.close()

### License (BSD 2-Clause)
Copyright (C) 2022 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE. 